# Training notebook

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from glob import glob
from collections import defaultdict

In [ ]:
# --- 1. Prepare Data ---
class KeypointDataset(Dataset):
    def __init__(self, data_dir):
        self.files = glob(os.path.join(data_dir, "*.npy"))  # Assuming data is stored as .npy files
        self.labels = [os.path.basename(f).split("_")[1] for f in self.files]  # Extract label from filename
        self.label_dict = {label: idx for idx, label in enumerate(sorted(set(self.labels)))}  # Create label mapping

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        keypoints = np.load(self.files[idx])
        keypoints = torch.tensor(keypoints, dtype=torch.float32)
        label = self.label_dict[self.labels[idx]]
        return keypoints, torch.tensor(label, dtype=torch.long)

In [ ]:
# --- 2. Create DataLoader ---
data_dir = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_gpu"
dataset = KeypointDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
# --- 3. Define LSTM Model ---
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        logits = self.fc(lstm_out.mean(dim=1))  # Average over all frames
        return torch.log_softmax(logits, dim=1)  # Log-softmax for stable gradients

# Model parameters
input_dim = dataset[0][0].shape[1]
hidden_dim = 256
output_dim = len(dataset.label_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_dim, hidden_dim, output_dim).to(device)

In [ ]:
# --- 4. Training & Validation with Early Stopping ---
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

def train_model(model, dataloader, criterion, optimizer, epochs=150, patience=10):
    model.train()
    best_loss = float("inf")
    epochs_no_improve = 0
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for keypoints, labels in dataloader:
            keypoints, labels = keypoints.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(keypoints)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        epoch_loss = running_loss / len(dataloader)
        epoch_accuracy = 100 * correct / total
        print(f"📌 Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%")
        
        # Early Stopping Check
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "best_lstm_model.pth")  # Save best model
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"🛑 Training stopped after {epoch+1} epochs due to no improvement.")
                break

train_model(model, dataloader, criterion, optimizer)

# New Training

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from glob import glob
import json

# --- 1. Load Labels ---
LABELS_PATH = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/label_to_index.json"
with open(LABELS_PATH, "r") as f:
    label_to_index = json.load(f)
index_to_label = {v: k for k, v in label_to_index.items()}

# --- 2. Prepare Data ---
class KeypointDataset(Dataset):
    def __init__(self, data_dir):
        self.files = glob(os.path.join(data_dir, "*.npy"))  # Load all keypoint files
        self.labels = [os.path.basename(f).split("_")[1] for f in self.files]  # Extract label from filename
        self.label_dict = label_to_index  # Use the same label mapping as in the model

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        keypoints = np.load(self.files[idx])
        keypoints = torch.tensor(keypoints, dtype=torch.float32)
        label = self.label_dict[self.labels[idx]]
        return keypoints, torch.tensor(label, dtype=torch.long)

# --- 3. Create DataLoader ---
data_dir = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_gpu"
dataset = KeypointDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# --- 4. Define LSTM Model ---
class LSTMModel(nn.Module):
    def __init__(self, input_dim=1662, hidden_dim=128, output_dim=len(label_to_index), num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out.mean(dim=1)) 
        return torch.log_softmax(out, dim=1)  # Log-softmax for stability

# --- 5. Initialize Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel().to(device)

# --- 6. Define Loss & Optimizer ---
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- 7. Training Loop with Early Stopping ---
epochs = 100
early_stopping_patience = 5  # Stop after 5 epochs with no improvement
best_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for keypoints, labels in dataloader:
        keypoints, labels = keypoints.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(keypoints)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    # Early Stopping Check
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0
        # Save best model
        MODEL_PATH = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/best_lstm_model.pth"
        torch.save(model.state_dict(), MODEL_PATH)
        print("✅ Model improved and saved!")
    else:
        patience_counter += 1
        print(f"⚠️ No improvement for {patience_counter} epochs.")
        if patience_counter >= early_stopping_patience:
            print("⏹ Early stopping triggered. Training stopped.")
            break

print("✅ Model training complete!")

FileNotFoundError: [Errno 2] No such file or directory: '/home/haggenmueller/asl_detection/machine_learning/models/lstm/label_to_index.json'